In [1]:
import torch
import torchvision
import numpy as np

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [5]:
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Linear Regression Model basic

In [6]:
w = torch.randn(2, 3, requires_grad = True)
b = torch.randn(2, requires_grad = True)

print(w, b)

tensor([[ 0.3018,  0.2814, -0.8400],
        [-0.1781,  1.4969,  0.6100]], requires_grad=True) tensor([1.0318, 0.1221], requires_grad=True)


In [7]:
def model(x):
    return x @ w.t() + b

In [8]:
preds = model(inputs)

print(f"Prediction {preds}", f"\n\nTargets {targets}")

Prediction tensor([[  5.7979, 113.6424],
        [ -0.5004, 154.6810],
        [ 16.2792, 220.5895],
        [ 12.8356,  68.8934],
        [ -9.9287, 174.2335]], grad_fn=<AddBackward0>) 

Targets tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Loss Function

In [9]:
# Mse loss

def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [10]:
# Compute loss

loss = mse(preds, targets)

print(loss)

tensor(4907.7158, grad_fn=<DivBackward0>)


In [11]:
# Compute gradient
loss.backward()

In [12]:
print(w)
print(w.grad)

tensor([[ 0.3018,  0.2814, -0.8400],
        [-0.1781,  1.4969,  0.6100]], requires_grad=True)
tensor([[-5748.9692, -7107.0771, -4315.3223],
        [ 4551.0786,  5211.7588,  3087.7598]])


In [13]:
# To reset grad value to zero before calling loss.backward again

w.grad.zero_()
b.grad.zero_()

print(w.grad,"\n",b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]]) 
 tensor([0., 0.])


In [19]:
# Generate predictions
loss = mse(preds, targets)
print(loss)

loss.backward()
preds = model(inputs)
print(preds)

tensor(4907.7158, grad_fn=<DivBackward0>)
tensor([[  5.7979, 113.6424],
        [ -0.5004, 154.6810],
        [ 16.2792, 220.5895],
        [ 12.8356,  68.8934],
        [ -9.9287, 174.2335]], grad_fn=<AddBackward0>)


In [20]:
# Adjust weights & reset gradients

with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [21]:
print(w)
print(b)

tensor([[ 0.3593,  0.3525, -0.7969],
        [-0.2236,  1.4448,  0.5791]], requires_grad=True)
tensor([1.0325, 0.1215], requires_grad=True)


In [22]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(3454.2129, grad_fn=<DivBackward0>)


In [23]:
# Train for 100 epochs
for i in range(500):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [24]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(39.2358, grad_fn=<DivBackward0>)


In [25]:
preds = model(inputs)

print(f"Prediction {preds}", f"\n\nTargets {targets}")

Prediction tensor([[ 58.3502,  70.4982],
        [ 76.6603,  98.2170],
        [129.4059, 138.2341],
        [ 26.3262,  37.7367],
        [ 89.4141, 114.4730]], grad_fn=<AddBackward0>) 

Targets tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Linear regression with pytorch built-in

In [27]:
import torch.nn as nn

In [28]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

# Dataset and Dataloader

In [30]:
from torch.utils.data import TensorDataset

In [34]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:2]

(tensor([[73., 67., 43.],
         [91., 88., 64.]]), tensor([[ 56.,  70.],
         [ 81., 101.]]))

In [36]:
from torch.utils.data import DataLoader

In [37]:
batch_size = 5

train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [38]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.]])
tensor([[103., 119.],
        [ 56.,  70.],
        [ 22.,  37.],
        [ 81., 101.],
        [ 22.,  37.]])


In [39]:
model = nn.Linear(3, 2) # nn.Linear(input_size, output_size)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1706, -0.0472,  0.4410],
        [-0.0807,  0.3012,  0.2203]], requires_grad=True)
Parameter containing:
tensor([0.3122, 0.3041], requires_grad=True)


In [40]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.1706, -0.0472,  0.4410],
         [-0.0807,  0.3012,  0.2203]], requires_grad=True),
 Parameter containing:
 tensor([0.3122, 0.3041], requires_grad=True)]

In [41]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 3.6601, 24.0707],
        [ 8.8594, 33.5706],
        [ 4.7250, 46.4287],
        [-2.8003, 13.1799],
        [14.8808, 39.0769],
        [ 3.6601, 24.0707],
        [ 8.8594, 33.5706],
        [ 4.7250, 46.4287],
        [-2.8003, 13.1799],
        [14.8808, 39.0769],
        [ 3.6601, 24.0707],
        [ 8.8594, 33.5706],
        [ 4.7250, 46.4287],
        [-2.8003, 13.1799],
        [14.8808, 39.0769]], grad_fn=<AddmmBackward>)

In [42]:
# Loss Function
import torch.nn.functional as F

In [43]:
loss_fn = F.mse_loss

In [44]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(5048.8481, grad_fn=<MseLossBackward>)


In [45]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [46]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [48]:
fit(500, model, loss_fn, opt, train_dl)

Epoch [10/500], Loss: 212.4721
Epoch [20/500], Loss: 63.2246
Epoch [30/500], Loss: 30.6564
Epoch [40/500], Loss: 63.9705
Epoch [50/500], Loss: 88.9227
Epoch [60/500], Loss: 26.9394
Epoch [70/500], Loss: 57.2927
Epoch [80/500], Loss: 24.3257
Epoch [90/500], Loss: 13.6757
Epoch [100/500], Loss: 16.4105
Epoch [110/500], Loss: 10.9340
Epoch [120/500], Loss: 8.9510
Epoch [130/500], Loss: 10.2236
Epoch [140/500], Loss: 7.1167
Epoch [150/500], Loss: 7.0430
Epoch [160/500], Loss: 2.8944
Epoch [170/500], Loss: 6.9261
Epoch [180/500], Loss: 4.4294
Epoch [190/500], Loss: 5.8207
Epoch [200/500], Loss: 3.3287
Epoch [210/500], Loss: 5.7640
Epoch [220/500], Loss: 5.2532
Epoch [230/500], Loss: 6.3352
Epoch [240/500], Loss: 4.8442
Epoch [250/500], Loss: 4.5646
Epoch [260/500], Loss: 3.2086
Epoch [270/500], Loss: 2.0326
Epoch [280/500], Loss: 4.8379
Epoch [290/500], Loss: 3.2178
Epoch [300/500], Loss: 1.6844
Epoch [310/500], Loss: 3.9389
Epoch [320/500], Loss: 1.8314
Epoch [330/500], Loss: 2.9814
Epoch 

In [49]:
preds = model(inputs)

print(f"Prediction {preds}", f"\n\nTargets {targets}")

Prediction tensor([[ 57.2001,  70.4317],
        [ 82.2397,  99.7821],
        [118.5138, 134.6120],
        [ 21.0223,  37.4684],
        [102.0769, 117.4010],
        [ 57.2001,  70.4317],
        [ 82.2397,  99.7821],
        [118.5138, 134.6120],
        [ 21.0223,  37.4684],
        [102.0769, 117.4010],
        [ 57.2001,  70.4317],
        [ 82.2397,  99.7821],
        [118.5138, 134.6120],
        [ 21.0223,  37.4684],
        [102.0769, 117.4010]], grad_fn=<AddmmBackward>) 

Targets tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
